# Habit Analysis - Direct SQLite Connection

In [ ]:
import pandas as pd
import sqlite3
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta

In [ ]:
# Connect directly to SQLite database
conn = sqlite3.connect('data/habits.db')

# Load and join data
df = pd.read_sql_query("""
SELECT 
    e.*,
    h.name as habit_name,
    h.unit,
    h.user_id
FROM entries e
JOIN habits h ON e.habit_id = h.id
ORDER BY e.date, e.timestamp
""", conn)

# Convert date and set index
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')

print(f"Loaded {len(df)} entries")
df.head()

In [ ]:
# Function to plot habits with different resolutions
def plot_habit(habit_id=None, habit_name=None, plot_type='sum', resolution='D'):
    """
    Plot habit data with different aggregations and resolutions
    
    Parameters:
    - habit_id: ID of the habit (optional if habit_name provided)
    - habit_name: Name of the habit (optional if habit_id provided)  
    - plot_type: 'sum', 'count', or 'average'
    - resolution: 'D' (daily), 'W' (weekly Mon-Sun), 'M' (monthly)
    """
    # Get habit data
    if habit_id:
        habit_data = df[df['habit_id'] == habit_id].copy()
        if habit_name is None and len(habit_data) > 0:
            habit_name = habit_data.iloc[0]['habit_name']
    else:
        habit_data = df[df['habit_name'] == habit_name].copy()
    
    if len(habit_data) == 0:
        print(f"No data found")
        return
    
    # Handle weekly resolution - use W-SUN for Monday-Sunday weeks
    if resolution == 'W' or resolution == 'W-MON':
        # Use W-SUN (week ending Sunday) for Monday-Sunday weeks
        grouped = habit_data['value'].resample('W-SUN')
    else:
        grouped = habit_data['value'].resample(resolution)
    
    if plot_type == 'sum':
        data = grouped.sum()
        title_suffix = 'Total'
    elif plot_type == 'count':
        data = grouped.count()
        title_suffix = 'Count'
    else:  # average
        data = grouped.sum() / grouped.count()
        title_suffix = 'Average'
    
    # Create resolution label
    res_label = {'D': 'Daily', 'W': 'Weekly', 'W-MON': 'Weekly', 'M': 'Monthly'}.get(resolution, resolution)
    
    # For weekly data, adjust labels to show week START date (Monday)
    if resolution == 'W' or resolution == 'W-MON':
        # W-SUN gives us Sunday dates, subtract 6 to get Monday start dates
        data.index = data.index - pd.Timedelta(days=6)
        
        # Check if the last week is the current (incomplete) week
        today = pd.Timestamp.now()
        is_current_week_incomplete = len(data) > 0 and today.dayofweek != 6  # Not Sunday
        
        # Get the Monday of the current week
        days_since_monday = today.dayofweek
        current_monday = today - pd.Timedelta(days=days_since_monday)
        
        if is_current_week_incomplete and len(data) > 0:
            # Check if last bar is current week
            is_current_week_incomplete = data.index[-1].date() == current_monday.date()
    
    # Plot
    fig = go.Figure()
    fig.add_trace(go.Bar(x=data.index, y=data.values))
    fig.update_layout(
        title=f'{habit_name} - {res_label} {title_suffix}',
        xaxis_title='Week Starting Monday' if resolution in ['W', 'W-MON'] else 'Date',
        yaxis_title=title_suffix
    )
    fig.show()
    
    # Print clarification for weekly data
    if resolution in ['W', 'W-MON'] and len(data) > 0:
        print(f"Note: Each bar represents Mon-Sun of that week")
        if 'is_current_week_incomplete' in locals() and is_current_week_incomplete:
            days_so_far = (today.dayofweek + 1) if today.dayofweek < 6 else 7
            print(f"Last bar (week of {data.index[-1].strftime('%b %d')}) has {days_so_far} days so far")
    
    return data

In [ ]:
# Examples of using the function
# Daily sum
plot_habit(habit_name='Zercher Squats', plot_type='sum', resolution='D')

# Weekly sum (weeks starting Monday)
plot_habit(habit_name='Zercher Squats', plot_type='sum', resolution='W')

# Weekly count
plot_habit(habit_name='Zercher Squats', plot_type='count', resolution='W')

# Weekly average
plot_habit(habit_name='Zercher Squats', plot_type='average', resolution='W')

In [ ]:
# Daily aggregations
daily_sum = habit_data['value'].resample('D').sum()
daily_count = habit_data['value'].resample('D').count()
daily_avg = daily_sum / daily_count  # True average

# Plot daily sum
fig = go.Figure()
fig.add_trace(go.Bar(x=daily_sum.index, y=daily_sum.values))
fig.update_layout(title=f'{habit_name} - Daily Total')
fig.show()

## Weekly Analysis

In [ ]:
# Weekly aggregations (calendar weeks starting Monday)
weekly_sum = habit_data['value'].resample('W-MON').sum()
weekly_count = habit_data['value'].resample('W-MON').count()
weekly_avg = weekly_sum / weekly_count

# Plot weekly sum
fig = go.Figure()
fig.add_trace(go.Bar(x=weekly_sum.index, y=weekly_sum.values))
fig.update_layout(title=f'{habit_name} - Weekly Total')
fig.show()

## Monthly Analysis

In [ ]:
# Monthly aggregations
monthly_sum = habit_data.resample('M')['value'].sum()
monthly_count = habit_data.resample('M')['value'].count()
monthly_avg = habit_data.resample('M')['value'].mean()

# Plot monthly trend
fig = go.Figure()
fig.add_trace(go.Scatter(x=monthly_sum.index, y=monthly_sum.values, mode='lines+markers', name='Monthly Total'))
fig.update_layout(title=f'{habit_name} - Monthly Trend', xaxis_title='Month', yaxis_title='Total')
fig.show()

## All Habits Overview

In [ ]:
# All habits comparison (last 30 days)
recent = df[df.index >= (datetime.now() - timedelta(days=30))]
habit_totals = recent.groupby('habit_name')['value'].agg(['sum', 'count', 'mean'])
habit_totals = habit_totals.sort_values('sum', ascending=False)

# Bar chart
fig = px.bar(habit_totals, y='sum', title='Habit Totals (Last 30 Days)')
fig.show()

habit_totals

## Custom Analysis Examples

In [ ]:
# Rolling 7-day average
habit_data['rolling_avg'] = habit_data['value'].rolling(window=7).mean()

fig = go.Figure()
fig.add_trace(go.Scatter(x=habit_data.index, y=habit_data['value'], mode='markers', name='Daily Values', opacity=0.5))
fig.add_trace(go.Scatter(x=habit_data.index, y=habit_data['rolling_avg'], mode='lines', name='7-Day Average'))
fig.update_layout(title=f'{habit_name} - 7-Day Rolling Average')
fig.show()

In [ ]:
# Day of week analysis
habit_data['day_of_week'] = habit_data.index.day_name()
dow_avg = habit_data.groupby('day_of_week')['value'].mean()

# Reorder days
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_avg = dow_avg.reindex(days_order)

fig = px.bar(x=dow_avg.index, y=dow_avg.values, title=f'{habit_name} - Average by Day of Week')
fig.show()